## Comparing observed and unobserved fishing characteristics in the drift gillnet fishery for swordfish: Linear Mixed Effects Model

Version: 10<br>
Date: updated 06/16/2022

Objective: Provide the Python code of the linear mixed-effects models that were used to compare trip and set levels for the different vessel groups and seasons. The two comparisons were (1) Unobserved trips and sets versus observed trips and sets by vessels that periodically carried observers to test the observer effect; and (2) Trips and sets on vessels that periodically carried observers versus trips and sets by unobservable vessels that were exempted from observer coverage or that had very low observer coverage to test the unobservable vessel effect. 

In [1]:
import cx_Oracle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf

import warnings
warnings.filterwarnings('ignore')

In [2]:
user = "xxx"
host = "xxxx"
port = "xxxx"
sid = "xxxx"
pswd = 'xxxx'
dsn = cx_Oracle.makedsn (host, port, sid)

connection  = cx_Oracle.connect (user, pswd, dsn)

## Mixed Effects Model: Trip Level Analysis

In [3]:
# Trip level metrics
sql = "SELECT * FROM VMS.DGN_TRIPVSVMS_V3" #VMS.DGN_TRIPVSVMS"
trips = pd.read_sql(sql, con=connection)

In [4]:
# converting depth longitude to positive values 
trips['AVG_DEPTH'] = trips['AVG_DEPTH'].abs()
trips['AVG_LONG'] = trips['AVG_LONG'].abs()
# converting vessel ID to a character field
trips['VESSEL_NUM'] = trips['VESSEL_NUM'].astype(str)

In [5]:
# excluding 2013 through Jan 2014, as well as when trip start date is null
tripsum = trips.loc[(trips.loc[:, 'IS_BAD_YEAR'] == 'F') & (trips.loc[:, 'IS_TRIP'] == 'T') & (trips.loc[:, 'IS_BAD_TRIP'] == 'F') & (trips.loc[:, 'PROB_VESSEL'] == 0), :]
# dropping rows where null in max landing date column
tripsum = tripsum.dropna(how='any', subset=['MAX_LANDING_DATE'])
tripsum = tripsum.reset_index(drop=True) 

In [6]:
# The high limit for acceptable values is the mean plus 2 standard deviations    
LimitHi = np.mean(tripsum.loc[:, 'HMS_LBS']) + 2*np.std(tripsum.loc[:, 'HMS_LBS'])
LimitLo = np.mean(tripsum.loc[:, 'HMS_LBS']) - 2*np.std(tripsum.loc[:, 'HMS_LBS'])
limits = (tripsum.loc[:, 'HMS_LBS'] >= LimitLo) & (tripsum.loc[:, 'HMS_LBS'] <= LimitHi)

In [7]:
# removing all hms lbs outside of 2 standard deviations
tripsumHMSLBS = tripsum[limits]

In [8]:
print(LimitHi)
print(LimitLo)

13651.516475256145
-5403.539508504227


In [9]:
LimitHiREV = np.mean(tripsum.loc[:, 'HMS_REVENUE']) + 2*np.std(tripsum.loc[:, 'HMS_REVENUE'])
LimitLoREV = np.mean(tripsum.loc[:, 'HMS_REVENUE']) - 2*np.std(tripsum.loc[:, 'HMS_REVENUE'])
limitsREV = (tripsum.loc[:, 'HMS_REVENUE'] >= LimitLoREV) & (tripsum.loc[:, 'HMS_REVENUE'] <= LimitHiREV)

In [10]:
print(LimitHiREV)
print(LimitLoREV)

24423.887801740784
-8127.709745219045


In [11]:
tripsumHMSLBSRev = tripsumHMSLBS[limitsREV]

In [12]:
tripsumHMSLBSRev = tripsumHMSLBSRev.reset_index(drop=True)

In [13]:
# log transform features
tripsumHMSLBSRev['HMS_CPUE_LOG'] = np.log(tripsumHMSLBSRev.loc[:,'HMS_CPUE'])
tripsumHMSLBSRev['HMS_LBS_LOG'] = np.log(tripsumHMSLBSRev.loc[:,'HMS_LBS'])
tripsumHMSLBSRev['HMS_REV_CPUE_LOG'] = np.log(tripsumHMSLBSRev.loc[:,'HMS_REV_CPUE'])
tripsumHMSLBSRev['CNT_SETS_LOG'] = np.log(tripsumHMSLBSRev.loc[:,'CNT_SETS'])
tripsumHMSLBSRev['AVG_DEPTH_LOG'] = np.log(tripsumHMSLBSRev.loc[:,'AVG_DEPTH'])
tripsumHMSLBSRev['AVG_DIST_SHORE_KM_LOG'] = np.log(tripsumHMSLBSRev.loc[:,'AVG_DIST_SHORE_KM'])
tripsumHMSLBSRev['PROPORTION_TRIP_FISHING_LOG'] = np.log(tripsumHMSLBSRev.loc[:,'PROPORTION_TRIP_FISHING'])
tripsumHMSLBSRev['HMS_REVENUE_LOG'] = np.log(tripsumHMSLBSRev.loc[:,'HMS_REVENUE'])
tripsumHMSLBSRev['TRIP_DIST_KM_LOG'] = np.log(tripsumHMSLBSRev.loc[:,'TRIP_DIST_KM'])
tripsumHMSLBSRev['TOTAL_TRIP_SECONDS_LOG'] = np.log(tripsumHMSLBSRev.loc[:,'TOTAL_TRIP_SECONDS'])
tripsumHMSLBSRev['AVG_TEMP_LOG'] = np.log(tripsumHMSLBSRev.loc[:,'AVG_TEMP'])
tripsumHMSLBSRev['AVG_LONG_LOG'] = np.log(tripsumHMSLBSRev.loc[:,'AVG_LONG'])
tripsumHMSLBSRev['AVG_LAT_LOG'] = np.log(tripsumHMSLBSRev.loc[:,'AVG_LAT'])

In [14]:
# can't use T and F in LME model correctly
#tripsumHMSLBSRev['IS_OBS_VESSEL'] = tripsumHMSLBSRev['IS_OBS_VESSEL']


#decoding for plot presentations
Replace = tripsumHMSLBSRev.loc[:, "IS_OBS_VESSEL"] == 'T'
tripsumHMSLBSRev.loc[Replace, "IS_OBS_VESSEL"] = 1

Replace = tripsumHMSLBSRev.loc[:, "IS_OBS_VESSEL"] == 'F'
tripsumHMSLBSRev.loc[Replace, "IS_OBS_VESSEL"] = 0


tripsumHMSLBSRev["IS_OBS_VESSEL"] = tripsumHMSLBSRev.loc[:, "IS_OBS_VESSEL"].astype(int)

In [15]:
# only vessels that carry observers, quality trips (excluding 2013), and season 1 
tripsum_obs_2013noS1 = tripsumHMSLBSRev.loc[(tripsumHMSLBSRev.loc[:,"IS_OBS_VESSEL"] == 1) & (tripsumHMSLBSRev.loc[:,"SEASON"] == 1), :]

# only vessels that carry observers, quality trips (excluding 2013), and season 2
tripsum_obs_2013noS2 = tripsumHMSLBSRev.loc[(tripsumHMSLBSRev.loc[:,"IS_OBS_VESSEL"] == 1) & (tripsumHMSLBSRev.loc[:,"SEASON"] == 2), :]


In [16]:
# vessels that carry observers vs vessels that don't, quality trips (excluding 2013), and season 1
tripsum_obsunobsS1 = tripsumHMSLBSRev.loc[(tripsumHMSLBSRev.loc[:,"SEASON"] == 1), :]

# vessels that carry observers vs vessels that don't, quality trips (excluding 2013), and season 2 
tripsum_obsunobsS2 = tripsumHMSLBSRev.loc[(tripsumHMSLBSRev.loc[:,"SEASON"] == 2), :]

In [17]:
print("\033[1m\n Counts of Observervations: Observed vs Unobserved, Season 1")
print(tripsum_obs_2013noS1.loc[(tripsum_obs_2013noS1.loc[:,"IS_OBSERVED_TRIP"] == 1), "VESSEL_NUM"].count())
print(tripsum_obs_2013noS1.loc[(tripsum_obs_2013noS1.loc[:,"IS_OBSERVED_TRIP"] == 0), "VESSEL_NUM"].count())


 Counts of Observervations: Observed vs Unobserved, Season 1
30
49


In [18]:
print("\033[1m\n Counts of Observervations: Observed vs Unobserved, Season 2")
print(tripsum_obs_2013noS2.loc[(tripsum_obs_2013noS2.loc[:,"IS_OBSERVED_TRIP"] == 1), "VESSEL_NUM"].count())
print(tripsum_obs_2013noS2.loc[(tripsum_obs_2013noS2.loc[:,"IS_OBSERVED_TRIP"] == 0), "VESSEL_NUM"].count())


 Counts of Observervations: Observed vs Unobserved, Season 2
45
128


In [19]:
print("\033[1m\n Counts of Observervations: Observable vs Unobservable, Season 1")
print(tripsum_obsunobsS1.loc[(tripsum_obsunobsS1.loc[:,"IS_OBS_VESSEL"] == 1), "VESSEL_NUM"].count())
print(tripsum_obsunobsS1.loc[(tripsum_obsunobsS1.loc[:,"IS_OBS_VESSEL"] == 0), "VESSEL_NUM"].count())


 Counts of Observervations: Observable vs Unobservable, Season 1
79
57


In [20]:
print("\033[1m\n Counts of Observervations: Observable vs Unobservable, Season 2")
print(tripsum_obsunobsS2.loc[(tripsum_obsunobsS2.loc[:,"IS_OBS_VESSEL"] == 1), "VESSEL_NUM"].count())
print(tripsum_obsunobsS2.loc[(tripsum_obsunobsS2.loc[:,"IS_OBS_VESSEL"] == 0), "VESSEL_NUM"].count())


 Counts of Observervations: Observable vs Unobservable, Season 2
173
55


### Average Latitude of HMS species

In [21]:
lat_md = smf.mixedlm("AVG_LAT_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
lat_mdf = lat_md.fit()
lat_coef = pd.DataFrame(lat_mdf.params, columns = ['Coef'])
lat_p = pd.DataFrame(lat_mdf.pvalues, columns = ['Pval']) # new
lat_se = pd.DataFrame(lat_mdf.bse, columns = ['StdErr']) # NEW
lat_ci = pd.DataFrame(lat_mdf.conf_int())
lat_nobs = pd.DataFrame({'col1': lat_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
lat_df = pd.concat([lat_coef, lat_se, lat_p, lat_ci, lat_nobs], axis=1) # NEW
lat_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
lat_df['SRC'] = 'Average Set Latitude'
lat_df.drop(['Intercept', 'Group Var'], inplace=True)

# obs vessels season 2
lat_md2 = smf.mixedlm("AVG_LAT_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
lat_mdf2 = lat_md2.fit()
lat_coef2 = pd.DataFrame(lat_mdf2.params, columns = ['Coef'])
lat_p2 = pd.DataFrame(lat_mdf2.pvalues, columns = ['Pval']) # new
lat_se2 = pd.DataFrame(lat_mdf2.bse, columns = ['StdErr']) # NEW
lat_ci2 = pd.DataFrame(lat_mdf2.conf_int())
lat_nobs2 = pd.DataFrame({'col1': lat_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
lat_df2 = pd.concat([lat_coef2, lat_se2, lat_p2, lat_ci2, lat_nobs2], axis=1) # NEW
lat_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
lat_df2['SRC'] = 'Average Set Latitude'
lat_df2.drop(['Intercept', 'Group Var'], inplace=True)


lat_mdx = smf.mixedlm("AVG_LAT_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
lat_mdfx = lat_mdx.fit()
lat_coefx = pd.DataFrame(lat_mdfx.params, columns = ['Coef'])
lat_px = pd.DataFrame(lat_mdfx.pvalues, columns = ['Pval']) # new
lat_sex = pd.DataFrame(lat_mdfx.bse, columns = ['StdErr']) # NEW
lat_cix = pd.DataFrame(lat_mdfx.conf_int())
lat_nobsx = pd.DataFrame({'col1': lat_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
lat_dfx = pd.concat([lat_coefx, lat_sex, lat_px, lat_cix, lat_nobsx], axis=1) # NEW
lat_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
lat_dfx['SRC'] = 'Average Set Latitude'
lat_dfx.drop(['Intercept', 'Group Var'], inplace=True)

lat_mdx2 = smf.mixedlm("AVG_LAT_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
lat_mdfx2 = lat_mdx2.fit()
lat_coefx2 = pd.DataFrame(lat_mdfx2.params, columns = ['Coef'])
lat_px2 = pd.DataFrame(lat_mdfx2.pvalues, columns = ['Pval']) # new
lat_sex2 = pd.DataFrame(lat_mdfx2.bse, columns = ['StdErr']) # NEW
lat_cix2 = pd.DataFrame(lat_mdfx2.conf_int())
lat_nobsx2 = pd.DataFrame({'col1': lat_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
lat_dfx2 = pd.concat([lat_coefx2, lat_sex2, lat_px2, lat_cix2, lat_nobsx2], axis=1) # NEW
lat_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
lat_dfx2 ['SRC'] = 'Average Set Latitude'
lat_dfx2.drop(['Intercept', 'Group Var'], inplace=True)

### Average Longitude of HMS species

In [22]:
long_md = smf.mixedlm("AVG_LONG_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
long_mdf = long_md.fit()
#print(long_mdf.summary())
long_coef = pd.DataFrame(long_mdf.params, columns = ['Coef'])
long_p = pd.DataFrame(long_mdf.pvalues, columns = ['Pval']) # new
long_se = pd.DataFrame(long_mdf.bse, columns = ['StdErr']) # NEW
long_ci = pd.DataFrame(long_mdf.conf_int())
long_nobs = pd.DataFrame({'col1': long_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
long_df = pd.concat([long_coef, long_se, long_p, long_ci, long_nobs], axis=1) # NEW
long_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
long_df['SRC'] = 'Average Set Longitude'
long_df.drop(['Intercept', 'Group Var'], inplace=True)

# obs vessels season 2
long_md2 = smf.mixedlm("AVG_LONG_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
long_mdf2 = long_md2.fit()
long_coef2 = pd.DataFrame(long_mdf2.params, columns = ['Coef'])
long_p2 = pd.DataFrame(long_mdf2.pvalues, columns = ['Pval']) # new
long_se2 = pd.DataFrame(long_mdf2.bse, columns = ['StdErr']) # NEW
long_ci2 = pd.DataFrame(long_mdf2.conf_int())
long_nobs2 = pd.DataFrame({'col1': long_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
long_df2 = pd.concat([long_coef2, long_se2, long_p2, long_ci2, long_nobs2], axis=1) # NEW
long_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
long_df2['SRC'] = 'Average Set Longitude'
long_df2.drop(['Intercept', 'Group Var'], inplace=True)

long_mdx = smf.mixedlm("AVG_LONG_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
long_mdfx = long_mdx.fit()
long_coefx = pd.DataFrame(long_mdfx.params, columns = ['Coef'])
long_px = pd.DataFrame(long_mdfx.pvalues, columns = ['Pval']) # new
long_sex = pd.DataFrame(long_mdfx.bse, columns = ['StdErr']) # NEW
long_cix = pd.DataFrame(long_mdfx.conf_int())
long_nobsx = pd.DataFrame({'col1': long_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
long_dfx = pd.concat([long_coefx, long_sex, long_px, long_cix, long_nobsx], axis=1) # NEW
long_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
long_dfx['SRC'] = 'Average Set Longitude'
long_dfx.drop(['Intercept', 'Group Var'], inplace=True)

long_mdx2 = smf.mixedlm("AVG_LONG_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
long_mdfx2 = long_mdx2.fit()
long_coefx2 = pd.DataFrame(long_mdfx2.params, columns = ['Coef'])
long_px2 = pd.DataFrame(long_mdfx2.pvalues, columns = ['Pval']) # new
long_sex2 = pd.DataFrame(long_mdfx2.bse, columns = ['StdErr']) # NEW
long_cix2 = pd.DataFrame(long_mdfx2.conf_int())
long_nobsx2 = pd.DataFrame({'col1': long_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
long_dfx2 = pd.concat([long_coefx2, long_sex2, long_px2, long_cix2, long_nobsx2], axis=1) # NEW
long_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
long_dfx2 ['SRC'] = 'Average Set Longitude'
long_dfx2.drop(['Intercept', 'Group Var'], inplace=True)

### Catch Per Unit of Effort (CPUE) of HMS species

In [23]:
# obs vessels season 1
cpue_md = smf.mixedlm("HMS_CPUE_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
cpue_mdf = cpue_md.fit()
#print(cpue_mdf.summary())
cpue_coef = pd.DataFrame(cpue_mdf.params, columns = ['Coef'])
cpue_p = pd.DataFrame(cpue_mdf.pvalues, columns = ['Pval']) # new
cpue_se = pd.DataFrame(cpue_mdf.bse, columns = ['StdErr']) # NEW
cpue_ci = pd.DataFrame(cpue_mdf.conf_int())
cpue_nobs = pd.DataFrame({'col1': cpue_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
cpue_df = pd.concat([cpue_coef, cpue_se, cpue_p, cpue_ci, cpue_nobs], axis=1) # NEW
cpue_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
cpue_df['SRC'] = 'CPUE'
cpue_df.drop(['Intercept', 'Group Var'], inplace=True)

# obs vessels season 2
cpue_md2 = smf.mixedlm("HMS_CPUE_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
cpue_mdf2 = cpue_md2.fit()
cpue_coef2 = pd.DataFrame(cpue_mdf2.params, columns = ['Coef'])
cpue_p2 = pd.DataFrame(cpue_mdf2.pvalues, columns = ['Pval']) # new
cpue_se2 = pd.DataFrame(cpue_mdf2.bse, columns = ['StdErr']) # NEW
cpue_ci2 = pd.DataFrame(cpue_mdf2.conf_int())
cpue_nobs2 = pd.DataFrame({'col1': cpue_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
cpue_df2 = pd.concat([cpue_coef2, cpue_se2, cpue_p2, cpue_ci2, cpue_nobs2], axis=1) # NEW
cpue_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
cpue_df2['SRC'] = 'CPUE'
cpue_df2.drop(['Intercept', 'Group Var'], inplace=True)

# observers vs vessels season 1
cpue_mdx = smf.mixedlm("HMS_CPUE_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
cpue_mdfx = cpue_mdx.fit()
cpue_coefx = pd.DataFrame(cpue_mdfx.params, columns = ['Coef'])
cpue_px = pd.DataFrame(cpue_mdfx.pvalues, columns = ['Pval']) # new
cpue_sex = pd.DataFrame(cpue_mdfx.bse, columns = ['StdErr']) # NEW
cpue_cix = pd.DataFrame(cpue_mdfx.conf_int())
cpue_nobsx = pd.DataFrame({'col1': cpue_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
cpue_dfx = pd.concat([cpue_coefx, cpue_sex, cpue_px, cpue_cix, cpue_nobsx], axis=1) # NEW
cpue_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
cpue_dfx['SRC'] = 'CPUE'
cpue_dfx.drop(['Intercept', 'Group Var'], inplace=True)

# observers vs vessels season 2
cpue_mdx2 = smf.mixedlm("HMS_CPUE_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
cpue_mdfx2 = cpue_mdx2.fit()
cpue_coefx2 = pd.DataFrame(cpue_mdfx2.params, columns = ['Coef'])
cpue_px2 = pd.DataFrame(cpue_mdfx2.pvalues, columns = ['Pval']) # new
cpue_sex2 = pd.DataFrame(cpue_mdfx2.bse, columns = ['StdErr']) # NEW
cpue_cix2 = pd.DataFrame(cpue_mdfx2.conf_int())
cpue_nobsx2 = pd.DataFrame({'col1': cpue_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
cpue_dfx2 = pd.concat([cpue_coefx2, cpue_sex2, cpue_px2, cpue_cix2, cpue_nobsx2], axis=1) # NEW
cpue_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
cpue_dfx2 ['SRC'] = 'CPUE'
cpue_dfx2.drop(['Intercept', 'Group Var'], inplace=True)


### Sold Lbs. to Shoreside Fish Processors of HMS species

In [24]:
# obs vessels all years
lbs_md = smf.mixedlm("HMS_LBS_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
lbs_mdf = lbs_md.fit()
#print(lbs_mdf.summary())
lbs_coef = pd.DataFrame(lbs_mdf.params, columns = ['Coef'])
lbs_p = pd.DataFrame(lbs_mdf.pvalues, columns = ['Pval']) # new
lbs_se = pd.DataFrame(lbs_mdf.bse, columns = ['StdErr']) # NEW
lbs_ci = pd.DataFrame(lbs_mdf.conf_int())
lbs_nobs = pd.DataFrame({'col1': lbs_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
lbs_df = pd.concat([lbs_coef, lbs_se, lbs_p, lbs_ci, lbs_nobs], axis=1) # NEW
lbs_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
lbs_df['SRC'] = 'Catch'
lbs_df.drop(['Intercept', 'Group Var'], inplace=True)

# obs vessels exclude 2013
lbs_md2 = smf.mixedlm("HMS_LBS_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
lbs_mdf2 = lbs_md2.fit()
lbs_coef2 = pd.DataFrame(lbs_mdf2.params, columns = ['Coef'])
lbs_p2 = pd.DataFrame(lbs_mdf2.pvalues, columns = ['Pval']) # new
lbs_se2 = pd.DataFrame(lbs_mdf2.bse, columns = ['StdErr']) # NEW
lbs_ci2 = pd.DataFrame(lbs_mdf2.conf_int())
lbs_nobs2 = pd.DataFrame({'col1': lbs_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
lbs_df2 = pd.concat([lbs_coef2, lbs_se2, lbs_p2, lbs_ci2, lbs_nobs2], axis=1) # NEW
lbs_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
lbs_df2['SRC'] = 'Catch'
lbs_df2.drop(['Intercept', 'Group Var'], inplace=True)

lbs_mdx = smf.mixedlm("HMS_LBS_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
lbs_mdfx = lbs_mdx.fit()
#print(lbs_mdfx.summary())
lbs_coefx = pd.DataFrame(lbs_mdfx.params, columns = ['Coef'])
lbs_px = pd.DataFrame(lbs_mdfx.pvalues, columns = ['Pval']) # new
lbs_sex = pd.DataFrame(lbs_mdfx.bse, columns = ['StdErr']) # NEW
lbs_cix = pd.DataFrame(lbs_mdfx.conf_int())
lbs_nobsx = pd.DataFrame({'col1': lbs_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
lbs_dfx = pd.concat([lbs_coefx, lbs_sex, lbs_px, lbs_cix, lbs_nobsx], axis=1) # NEW
lbs_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
lbs_dfx['SRC'] = 'Catch'
lbs_dfx.drop(['Intercept', 'Group Var'], inplace=True)

lbs_mdx2 = smf.mixedlm("HMS_LBS_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
lbs_mdfx2 = lbs_mdx2.fit()
#print(lbs_mdfx.summary())
lbs_coefx2 = pd.DataFrame(lbs_mdfx2.params, columns = ['Coef'])
lbs_px2 = pd.DataFrame(lbs_mdfx2.pvalues, columns = ['Pval']) # new
lbs_sex2 = pd.DataFrame(lbs_mdfx2.bse, columns = ['StdErr']) # NEW
lbs_cix2 = pd.DataFrame(lbs_mdfx2.conf_int())
lbs_nobsx2 = pd.DataFrame({'col1': lbs_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
lbs_dfx2 = pd.concat([lbs_coefx2, lbs_sex2, lbs_px2, lbs_cix2, lbs_nobsx2], axis=1) # NEW
lbs_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
lbs_dfx2['SRC'] = 'Catch'
lbs_dfx2.drop(['Intercept', 'Group Var'], inplace=True)

### Ex-vessel Revenue Per Unit of Effort (RPUE) of HMS Species

In [25]:
# obs vessels all years
rpue_md = smf.mixedlm("HMS_REV_CPUE_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
rpue_mdf = rpue_md.fit()
#print(rpue_mdf.summary())
rpue_coef = pd.DataFrame(rpue_mdf.params, columns = ['Coef'])
rpue_p = pd.DataFrame(rpue_mdf.pvalues, columns = ['Pval']) # new
rpue_se = pd.DataFrame(rpue_mdf.bse, columns = ['StdErr']) # NEW
rpue_ci = pd.DataFrame(rpue_mdf.conf_int())
rpue_nobs = pd.DataFrame({'col1': rpue_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
rpue_df = pd.concat([rpue_coef, rpue_se, rpue_p, rpue_ci, rpue_nobs], axis=1) # NEW
rpue_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
rpue_df['SRC'] = 'RPUE'
rpue_df.drop(['Intercept', 'Group Var'], inplace=True)

# obs vessels exclude 2013
rpue_md2 = smf.mixedlm("HMS_REV_CPUE_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
rpue_mdf2 = rpue_md2.fit()
rpue_coef2 = pd.DataFrame(rpue_mdf2.params, columns = ['Coef'])
rpue_p2 = pd.DataFrame(rpue_mdf2.pvalues, columns = ['Pval']) # new
rpue_se2 = pd.DataFrame(rpue_mdf2.bse, columns = ['StdErr']) # NEW
rpue_ci2 = pd.DataFrame(rpue_mdf2.conf_int())
rpue_nobs2 = pd.DataFrame({'col1': rpue_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
rpue_df2 = pd.concat([rpue_coef2, rpue_se2, rpue_p2, rpue_ci2, rpue_nobs2], axis=1) # NEW
rpue_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
rpue_df2['SRC'] = 'RPUE'
rpue_df2.drop(['Intercept', 'Group Var'], inplace=True)

rpue_mdx = smf.mixedlm("HMS_REV_CPUE_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
rpue_mdfx = rpue_mdx.fit()
rpue_coefx = pd.DataFrame(rpue_mdfx.params, columns = ['Coef'])
rpue_px = pd.DataFrame(rpue_mdfx.pvalues, columns = ['Pval']) # new
rpue_sex = pd.DataFrame(rpue_mdfx.bse, columns = ['StdErr']) # NEW
rpue_cix = pd.DataFrame(rpue_mdfx.conf_int())
rpue_nobsx = pd.DataFrame({'col1': rpue_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
rpue_dfx = pd.concat([rpue_coefx, rpue_sex, rpue_px, rpue_cix, rpue_nobsx], axis=1) # NEW
rpue_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
rpue_dfx['SRC'] = 'RPUE'
rpue_dfx.drop(['Intercept', 'Group Var'], inplace=True)

rpue_mdx2 = smf.mixedlm("HMS_REV_CPUE_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
rpue_mdfx2 = rpue_mdx2.fit()
rpue_coefx2 = pd.DataFrame(rpue_mdfx2.params, columns = ['Coef'])
rpue_px2 = pd.DataFrame(rpue_mdfx2.pvalues, columns = ['Pval']) # new
rpue_sex2 = pd.DataFrame(rpue_mdfx2.bse, columns = ['StdErr']) # NEW
rpue_cix2 = pd.DataFrame(rpue_mdfx2.conf_int())
rpue_nobsx2 = pd.DataFrame({'col1': rpue_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
rpue_dfx2 = pd.concat([rpue_coefx2, rpue_sex2, rpue_px2, rpue_cix2, rpue_nobsx2], axis=1) # NEW
rpue_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
rpue_dfx2['SRC'] = 'RPUE'
rpue_dfx2.drop(['Intercept', 'Group Var'], inplace=True)

### Number of Fishing Sets (i.e., Fishing Effort)

In [26]:
set_md = smf.mixedlm("CNT_SETS_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
set_mdf = set_md.fit()
#print(set_mdf.summary())
set_coef = pd.DataFrame(set_mdf.params, columns = ['Coef'])
set_p = pd.DataFrame(set_mdf.pvalues, columns = ['Pval']) # new
set_se = pd.DataFrame(set_mdf.bse, columns = ['StdErr']) # NEW
set_ci = pd.DataFrame(set_mdf.conf_int())
set_nobs = pd.DataFrame({'col1': set_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
set_df = pd.concat([set_coef, set_se, set_p, set_ci, set_nobs], axis=1) # NEW
set_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
set_df['SRC'] = 'Effort'
set_df.drop(['Intercept', 'Group Var'], inplace=True)

set_md2 = smf.mixedlm("CNT_SETS_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
set_mdf2 = set_md2.fit()
set_coef2 = pd.DataFrame(set_mdf2.params, columns = ['Coef'])
set_p2 = pd.DataFrame(set_mdf2.pvalues, columns = ['Pval']) # new
set_se2 = pd.DataFrame(set_mdf2.bse, columns = ['StdErr']) # NEW
set_ci2 = pd.DataFrame(set_mdf2.conf_int())
set_nobs2 = pd.DataFrame({'col1': set_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
set_df2 = pd.concat([set_coef2, set_se2, set_p2, set_ci2, set_nobs2], axis=1) # NEW
set_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
set_df2['SRC'] = 'Effort'
set_df2.drop(['Intercept', 'Group Var'], inplace=True)

set_mdx = smf.mixedlm("CNT_SETS_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
set_mdfx = set_mdx.fit()
set_coefx = pd.DataFrame(set_mdfx.params, columns = ['Coef'])
set_px = pd.DataFrame(set_mdfx.pvalues, columns = ['Pval']) # new
set_sex = pd.DataFrame(set_mdfx.bse, columns = ['StdErr']) # NEW
set_cix = pd.DataFrame(set_mdfx.conf_int())
set_nobsx = pd.DataFrame({'col1': set_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
set_dfx = pd.concat([set_coefx, set_sex, set_px, set_cix, set_nobsx], axis=1) # NEW
set_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
set_dfx['SRC'] = 'Effort'
set_dfx.drop(['Intercept', 'Group Var'], inplace=True)

set_mdx2 = smf.mixedlm("CNT_SETS_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
set_mdfx2 = set_mdx2.fit()
set_coefx2 = pd.DataFrame(set_mdfx2.params, columns = ['Coef'])
set_px2 = pd.DataFrame(set_mdfx2.pvalues, columns = ['Pval']) # new
set_sex2 = pd.DataFrame(set_mdfx2.bse, columns = ['StdErr']) # NEW
set_cix2 = pd.DataFrame(set_mdfx2.conf_int())
set_nobsx2 = pd.DataFrame({'col1': set_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
set_dfx2 = pd.concat([set_coefx2, set_sex2, set_px2, set_cix2, set_nobsx2], axis=1) # NEW
set_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
set_dfx2['SRC'] = 'Effort'
set_dfx2.drop(['Intercept', 'Group Var'], inplace=True)

### Average Bottom Depth during Fishing Sets

In [27]:
dep_md = smf.mixedlm("AVG_DEPTH_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
dep_mdf = dep_md.fit()
#print(dep_mdf.summary())
dep_coef = pd.DataFrame(dep_mdf.params, columns = ['Coef'])
dep_p = pd.DataFrame(dep_mdf.pvalues, columns = ['Pval']) # new
dep_se = pd.DataFrame(dep_mdf.bse, columns = ['StdErr']) # NEW
dep_ci = pd.DataFrame(dep_mdf.conf_int())
dep_nobs = pd.DataFrame({'col1': dep_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
dep_df = pd.concat([dep_coef, dep_se, dep_p, dep_ci, dep_nobs], axis=1) # NEW
dep_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
dep_df['SRC'] = 'Average Set Depth'
dep_df.drop(['Intercept', 'Group Var'], inplace=True)

dep_md2 = smf.mixedlm("AVG_DEPTH_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
dep_mdf2 = dep_md2.fit()
dep_coef2 = pd.DataFrame(dep_mdf2.params, columns = ['Coef'])
dep_p2 = pd.DataFrame(dep_mdf2.pvalues, columns = ['Pval']) # new
dep_se2 = pd.DataFrame(dep_mdf2.bse, columns = ['StdErr']) # NEW
dep_ci2 = pd.DataFrame(dep_mdf2.conf_int())
dep_nobs2 = pd.DataFrame({'col1': dep_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
dep_df2 = pd.concat([dep_coef2, dep_se2, dep_p2, dep_ci2, dep_nobs2], axis=1) # NEW
dep_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
dep_df2['SRC'] = 'Average Set Depth'
dep_df2.drop(['Intercept', 'Group Var'], inplace=True)

dep_mdx = smf.mixedlm("AVG_DEPTH_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
dep_mdfx = dep_mdx.fit()
dep_coefx = pd.DataFrame(dep_mdfx.params, columns = ['Coef'])
dep_px = pd.DataFrame(dep_mdfx.pvalues, columns = ['Pval']) # new
dep_sex = pd.DataFrame(dep_mdfx.bse, columns = ['StdErr']) # NEW
dep_cix = pd.DataFrame(dep_mdfx.conf_int())
dep_nobsx = pd.DataFrame({'col1': dep_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
dep_dfx = pd.concat([dep_coefx, dep_sex, dep_px, dep_cix, dep_nobsx], axis=1) # NEW
dep_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
dep_dfx['SRC'] = 'Average Set Depth'
dep_dfx.drop(['Intercept', 'Group Var'], inplace=True)


dep_mdx2 = smf.mixedlm("AVG_DEPTH_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
dep_mdfx2 = dep_mdx2.fit()
dep_coefx2 = pd.DataFrame(dep_mdfx2.params, columns = ['Coef'])
dep_px2 = pd.DataFrame(dep_mdfx2.pvalues, columns = ['Pval']) # new
dep_sex2 = pd.DataFrame(dep_mdfx2.bse, columns = ['StdErr']) # NEW
dep_cix2 = pd.DataFrame(dep_mdfx2.conf_int())
dep_nobsx2 = pd.DataFrame({'col1': dep_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
dep_dfx2 = pd.concat([dep_coefx2, dep_sex2, dep_px2, dep_cix2, dep_nobsx2], axis=1) # NEW
dep_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
dep_dfx2['SRC'] = 'Average Set Depth'
dep_dfx2.drop(['Intercept', 'Group Var'], inplace=True)

### Average Distance from Shore during Fishing Sets

In [28]:
distshr_md = smf.mixedlm("AVG_DIST_SHORE_KM_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
distshr_mdf = distshr_md.fit()
#print(distshr_mdf.summary())
distshr_coef = pd.DataFrame(distshr_mdf.params, columns = ['Coef'])
distshr_p = pd.DataFrame(distshr_mdf.pvalues, columns = ['Pval']) # new
distshr_se = pd.DataFrame(distshr_mdf.bse, columns = ['StdErr']) # NEW
distshr_ci = pd.DataFrame(distshr_mdf.conf_int())
distshr_nobs = pd.DataFrame({'col1': distshr_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
distshr_df = pd.concat([distshr_coef, distshr_se, distshr_p, distshr_ci, distshr_nobs], axis=1) # NEW
distshr_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
distshr_df['SRC'] = 'Average Set Distance from Shore'
distshr_df.drop(['Intercept', 'Group Var'], inplace=True)

distshr_md2 = smf.mixedlm("AVG_DIST_SHORE_KM_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
distshr_mdf2 = distshr_md2.fit()
#print(distshr_mdf2.summary())
distshr_coef2 = pd.DataFrame(distshr_mdf2.params, columns = ['Coef'])
distshr_p2 = pd.DataFrame(distshr_mdf2.pvalues, columns = ['Pval']) # new
distshr_se2 = pd.DataFrame(distshr_mdf2.bse, columns = ['StdErr']) # NEW
distshr_ci2 = pd.DataFrame(distshr_mdf2.conf_int())
distshr_nobs2 = pd.DataFrame({'col1': distshr_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
distshr_df2 = pd.concat([distshr_coef2, distshr_se2, distshr_p2, distshr_ci2, distshr_nobs2], axis=1) # NEW
distshr_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
distshr_df2['SRC'] = 'Average Set Distance from Shore'
distshr_df2.drop(['Intercept', 'Group Var'], inplace=True)

distshr_mdx = smf.mixedlm("AVG_DIST_SHORE_KM_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
distshr_mdfx = distshr_mdx.fit()
#print(distshr_mdf2.summary())
distshr_coefx = pd.DataFrame(distshr_mdfx.params, columns = ['Coef'])
distshr_px = pd.DataFrame(distshr_mdfx.pvalues, columns = ['Pval']) # new
distshr_sex = pd.DataFrame(distshr_mdfx.bse, columns = ['StdErr']) # NEW
distshr_cix = pd.DataFrame(distshr_mdfx.conf_int())
distshr_nobsx = pd.DataFrame({'col1': distshr_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
distshr_dfx = pd.concat([distshr_coefx, distshr_sex, distshr_px, distshr_cix, distshr_nobsx], axis=1) # NEW
distshr_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
distshr_dfx['SRC'] = 'Average Set Distance from Shore'
distshr_dfx.drop(['Intercept', 'Group Var'], inplace=True)

distshr_mdx2 = smf.mixedlm("AVG_DIST_SHORE_KM_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
distshr_mdfx2 = distshr_mdx2.fit()
#print(distshr_mdf2.summary())
distshr_coefx2 = pd.DataFrame(distshr_mdfx2.params, columns = ['Coef'])
distshr_px2 = pd.DataFrame(distshr_mdfx2.pvalues, columns = ['Pval']) # new
distshr_sex2 = pd.DataFrame(distshr_mdfx2.bse, columns = ['StdErr']) # NEW
distshr_cix2 = pd.DataFrame(distshr_mdfx2.conf_int())
distshr_nobsx2 = pd.DataFrame({'col1': distshr_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
distshr_dfx2 = pd.concat([distshr_coefx2, distshr_sex2, distshr_px2, distshr_cix2, distshr_nobsx2], axis=1) # NEW
distshr_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
distshr_dfx2['SRC'] = 'Average Set Distance from Shore'
distshr_dfx2.drop(['Intercept', 'Group Var'], inplace=True)

### Proportion of At-sea Trips that were Classified as Fishing

In [29]:
pptf_md = smf.mixedlm("PROPORTION_TRIP_FISHING_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
pptf_mdf = pptf_md.fit()
#print(pptf_mdf.summary())
pptf_coef = pd.DataFrame(pptf_mdf.params, columns = ['Coef'])
pptf_p = pd.DataFrame(pptf_mdf.pvalues, columns = ['Pval']) # new
pptf_se = pd.DataFrame(pptf_mdf.bse, columns = ['StdErr']) # NEW
pptf_ci = pd.DataFrame(pptf_mdf.conf_int())
pptf_nobs = pd.DataFrame({'col1': pptf_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
pptf_df = pd.concat([pptf_coef, pptf_se, pptf_p, pptf_ci, pptf_nobs], axis=1) # NEW
pptf_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
pptf_df['SRC'] = 'Proportion of Trip Fishing'
pptf_df.drop(['Intercept', 'Group Var'], inplace=True)

pptf_md2 = smf.mixedlm("PROPORTION_TRIP_FISHING_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
pptf_mdf2 = pptf_md2.fit()
pptf_coef2 = pd.DataFrame(pptf_mdf2.params, columns = ['Coef'])
pptf_p2 = pd.DataFrame(pptf_mdf2.pvalues, columns = ['Pval']) # new
pptf_se2 = pd.DataFrame(pptf_mdf2.bse, columns = ['StdErr']) # NEW
pptf_ci2 = pd.DataFrame(pptf_mdf2.conf_int())
pptf_nobs2 = pd.DataFrame({'col1': pptf_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
pptf_df2 = pd.concat([pptf_coef2, pptf_se2, pptf_p2, pptf_ci2, pptf_nobs2], axis=1) # NEW
pptf_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
pptf_df2['SRC'] = 'Proportion of Trip Fishing'
pptf_df2.drop(['Intercept', 'Group Var'], inplace=True)

pptf_mdx = smf.mixedlm("PROPORTION_TRIP_FISHING_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
pptf_mdfx = pptf_mdx.fit()
pptf_coefx = pd.DataFrame(pptf_mdfx.params, columns = ['Coef'])
pptf_px = pd.DataFrame(pptf_mdfx.pvalues, columns = ['Pval']) # new
pptf_sex = pd.DataFrame(pptf_mdfx.bse, columns = ['StdErr']) # NEW
pptf_cix = pd.DataFrame(pptf_mdfx.conf_int())
pptf_nobsx = pd.DataFrame({'col1': pptf_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
pptf_dfx = pd.concat([pptf_coefx, pptf_sex, pptf_px, pptf_cix, pptf_nobsx], axis=1) # NEW
pptf_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
pptf_dfx['SRC'] = 'Proportion of Trip Fishing'
pptf_dfx.drop(['Intercept', 'Group Var'], inplace=True)

pptf_mdx2 = smf.mixedlm("PROPORTION_TRIP_FISHING_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
pptf_mdfx2 = pptf_mdx2.fit()
pptf_coefx2 = pd.DataFrame(pptf_mdfx2.params, columns = ['Coef'])
pptf_px2 = pd.DataFrame(pptf_mdfx2.pvalues, columns = ['Pval']) # new
pptf_sex2 = pd.DataFrame(pptf_mdfx2.bse, columns = ['StdErr']) # NEW
pptf_cix2 = pd.DataFrame(pptf_mdfx2.conf_int())
pptf_nobsx2 = pd.DataFrame({'col1': pptf_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
pptf_dfx2 = pd.concat([pptf_coefx2, pptf_sex2, pptf_px2, pptf_cix2, pptf_nobsx2], axis=1) # NEW
pptf_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
pptf_dfx2['SRC'] = 'Proportion of Trip Fishing'
pptf_dfx2.drop(['Intercept', 'Group Var'], inplace=True)

### Ex-vessel Revenue of HMS species Sold to Shoreside Processors

In [30]:
rev_md = smf.mixedlm("HMS_REVENUE_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
rev_mdf = rev_md.fit()
#print(rev_mdf.summary())
rev_coef = pd.DataFrame(rev_mdf.params, columns = ['Coef'])
rev_p = pd.DataFrame(rev_mdf.pvalues, columns = ['Pval']) # new
rev_se = pd.DataFrame(rev_mdf.bse, columns = ['StdErr']) # NEW
rev_ci = pd.DataFrame(rev_mdf.conf_int())
rev_nobs = pd.DataFrame({'col1': rev_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
rev_df = pd.concat([rev_coef, rev_se, rev_p, rev_ci, rev_nobs], axis=1) # NEW
rev_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
rev_df['SRC'] = 'Ex-vessel Revenue'
rev_df.drop(['Intercept', 'Group Var'], inplace=True)

rev_md2 = smf.mixedlm("HMS_REVENUE_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
rev_mdf2 = rev_md2.fit()
rev_coef2 = pd.DataFrame(rev_mdf2.params, columns = ['Coef'])
rev_p2 = pd.DataFrame(rev_mdf2.pvalues, columns = ['Pval']) # new
rev_se2 = pd.DataFrame(rev_mdf2.bse, columns = ['StdErr']) # NEW
rev_ci2 = pd.DataFrame(rev_mdf2.conf_int())
rev_nobs2 = pd.DataFrame({'col1': rev_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
rev_df2 = pd.concat([rev_coef2, rev_se2, rev_p2, rev_ci2, rev_nobs2], axis=1) # NEW
rev_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
rev_df2['SRC'] = 'Ex-vessel Revenue'
rev_df2.drop(['Intercept', 'Group Var'], inplace=True)

rev_mdx = smf.mixedlm("HMS_REVENUE_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
rev_mdfx = rev_mdx.fit()
rev_coefx = pd.DataFrame(rev_mdfx.params, columns = ['Coef'])
rev_px = pd.DataFrame(rev_mdfx.pvalues, columns = ['Pval']) # new
rev_sex = pd.DataFrame(rev_mdfx.bse, columns = ['StdErr']) # NEW
rev_cix = pd.DataFrame(rev_mdfx.conf_int())
rev_nobsx = pd.DataFrame({'col1': rev_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
rev_dfx = pd.concat([rev_coefx, rev_sex, rev_px, rev_cix, rev_nobsx], axis=1) # NEW
rev_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
rev_dfx['SRC'] = 'Ex-vessel Revenue'
rev_dfx.drop(['Intercept', 'Group Var'], inplace=True)

rev_mdx2 = smf.mixedlm("HMS_REVENUE_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
rev_mdfx2 = rev_mdx2.fit()
rev_coefx2 = pd.DataFrame(rev_mdfx2.params, columns = ['Coef'])
rev_px2 = pd.DataFrame(rev_mdfx2.pvalues, columns = ['Pval']) # new
rev_sex2 = pd.DataFrame(rev_mdfx2.bse, columns = ['StdErr']) # NEW
rev_cix2 = pd.DataFrame(rev_mdfx2.conf_int())
rev_nobsx2 = pd.DataFrame({'col1': rev_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
rev_dfx2 = pd.concat([rev_coefx2, rev_sex2, rev_px2, rev_cix2, rev_nobsx2], axis=1) # NEW
rev_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
rev_dfx2['SRC'] = 'Ex-vessel Revenue'
rev_dfx2.drop(['Intercept', 'Group Var'], inplace=True)

### Total Fishing Trip Distance From Departure Port to Arrival Port

In [31]:
trpdist_md = smf.mixedlm("TRIP_DIST_KM_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
trpdist_mdf = trpdist_md.fit()
#print(trpdist_mdf.summary())
trpdist_coef = pd.DataFrame(trpdist_mdf.params, columns = ['Coef'])
trpdist_p = pd.DataFrame(trpdist_mdf.pvalues, columns = ['Pval']) # new
trpdist_se = pd.DataFrame(trpdist_mdf.bse, columns = ['StdErr']) # NEW
trpdist_ci = pd.DataFrame(trpdist_mdf.conf_int())
trpdist_nobs = pd.DataFrame({'col1': trpdist_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
trpdist_df = pd.concat([trpdist_coef, trpdist_se, trpdist_p, trpdist_ci, trpdist_nobs], axis=1) # NEW
trpdist_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
trpdist_df['SRC'] = 'Trip Distance'
trpdist_df.drop(['Intercept', 'Group Var'], inplace=True)

trpdist_md2 = smf.mixedlm("TRIP_DIST_KM_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
trpdist_mdf2 = trpdist_md2.fit()
trpdist_coef2 = pd.DataFrame(trpdist_mdf2.params, columns = ['Coef'])
trpdist_p2 = pd.DataFrame(trpdist_mdf2.pvalues, columns = ['Pval']) # new
trpdist_se2 = pd.DataFrame(trpdist_mdf2.bse, columns = ['StdErr']) # NEW
trpdist_ci2 = pd.DataFrame(trpdist_mdf2.conf_int())
trpdist_nobs2 = pd.DataFrame({'col1': trpdist_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
trpdist_df2 = pd.concat([trpdist_coef2, trpdist_se2, trpdist_p2, trpdist_ci2, trpdist_nobs2], axis=1) # NEW
trpdist_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
trpdist_df2['SRC'] = 'Trip Distance'
trpdist_df2.drop(['Intercept', 'Group Var'], inplace=True)

trpdist_mdx = smf.mixedlm("TRIP_DIST_KM_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
trpdist_mdfx = trpdist_mdx.fit()
#print(trpdist_mdfx.summary())
trpdist_coefx = pd.DataFrame(trpdist_mdfx.params, columns = ['Coef'])
trpdist_px = pd.DataFrame(trpdist_mdfx.pvalues, columns = ['Pval']) # new
trpdist_sex = pd.DataFrame(trpdist_mdfx.bse, columns = ['StdErr']) # NEW
trpdist_cix = pd.DataFrame(trpdist_mdfx.conf_int())
trpdist_nobsx = pd.DataFrame({'col1': trpdist_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
trpdist_dfx = pd.concat([trpdist_coefx, trpdist_sex, trpdist_px, trpdist_cix, trpdist_nobsx], axis=1) # NEW
trpdist_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
trpdist_dfx['SRC'] = 'Trip Distance'
trpdist_dfx.drop(['Intercept', 'Group Var'], inplace=True)

trpdist_mdx2 = smf.mixedlm("TRIP_DIST_KM_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
trpdist_mdfx2 = trpdist_mdx2.fit()
#print(trpdist_mdfx.summary())
trpdist_coefx2 = pd.DataFrame(trpdist_mdfx2.params, columns = ['Coef'])
trpdist_px2 = pd.DataFrame(trpdist_mdfx2.pvalues, columns = ['Pval']) # new
trpdist_sex2 = pd.DataFrame(trpdist_mdfx2.bse, columns = ['StdErr']) # NEW
trpdist_cix2 = pd.DataFrame(trpdist_mdfx2.conf_int())
trpdist_nobsx2 = pd.DataFrame({'col1': trpdist_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
trpdist_dfx2 = pd.concat([trpdist_coefx2, trpdist_sex2, trpdist_px2, trpdist_cix2, trpdist_nobsx2], axis=1) # NEW
trpdist_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
trpdist_dfx2['SRC'] = 'Trip Distance'
trpdist_dfx2.drop(['Intercept', 'Group Var'], inplace=True)

### Total Fishing Trip Duration From Departure Port to Arrival Port

In [32]:
tts_md = smf.mixedlm("TOTAL_TRIP_SECONDS_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
tts_mdf = tts_md.fit()
#print(tts_mdf.summary())
tts_coef = pd.DataFrame(tts_mdf.params, columns = ['Coef'])
tts_p = pd.DataFrame(tts_mdf.pvalues, columns = ['Pval']) # new
tts_se = pd.DataFrame(tts_mdf.bse, columns = ['StdErr']) # NEW
tts_ci = pd.DataFrame(tts_mdf.conf_int())
tts_nobs = pd.DataFrame({'col1': tts_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
tts_df = pd.concat([tts_coef, tts_se, tts_p, tts_ci, tts_nobs], axis=1) # NEW
tts_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
tts_df['SRC'] = 'Trip Duration'
tts_df.drop(['Intercept', 'Group Var'], inplace=True)

tts_md2 = smf.mixedlm("TOTAL_TRIP_SECONDS_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
tts_mdf2 = tts_md2.fit()
tts_coef2 = pd.DataFrame(tts_mdf2.params, columns = ['Coef'])
tts_p2 = pd.DataFrame(tts_mdf2.pvalues, columns = ['Pval']) # new
tts_se2 = pd.DataFrame(tts_mdf2.bse, columns = ['StdErr']) # NEW
tts_ci2 = pd.DataFrame(tts_mdf2.conf_int())
tts_nobs2 = pd.DataFrame({'col1': tts_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
tts_df2 = pd.concat([tts_coef2, tts_se2, tts_p2, tts_ci2, tts_nobs2], axis=1) # NEW
tts_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
tts_df2['SRC'] = 'Trip Duration'
tts_df2.drop(['Intercept', 'Group Var'], inplace=True)

tts_mdx = smf.mixedlm("TOTAL_TRIP_SECONDS_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
tts_mdfx = tts_mdx.fit()
tts_coefx = pd.DataFrame(tts_mdfx.params, columns = ['Coef'])
tts_px = pd.DataFrame(tts_mdfx.pvalues, columns = ['Pval']) # new
tts_sex = pd.DataFrame(tts_mdfx.bse, columns = ['StdErr']) # NEW
tts_cix = pd.DataFrame(tts_mdfx.conf_int())
tts_nobsx = pd.DataFrame({'col1': tts_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
tts_dfx = pd.concat([tts_coefx, tts_sex, tts_px, tts_cix, tts_nobsx], axis=1) # NEW
tts_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
tts_dfx['SRC'] = 'Trip Duration'
tts_dfx.drop(['Intercept', 'Group Var'], inplace=True)

tts_mdx2 = smf.mixedlm("TOTAL_TRIP_SECONDS_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
tts_mdfx2 = tts_mdx2.fit()
tts_coefx2 = pd.DataFrame(tts_mdfx2.params, columns = ['Coef'])
tts_px2 = pd.DataFrame(tts_mdfx2.pvalues, columns = ['Pval']) # new
tts_sex2 = pd.DataFrame(tts_mdfx2.bse, columns = ['StdErr']) # NEW
tts_cix2 = pd.DataFrame(tts_mdfx2.conf_int())
tts_nobsx2 = pd.DataFrame({'col1': tts_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
tts_dfx2 = pd.concat([tts_coefx2, tts_sex2, tts_px2, tts_cix2, tts_nobsx2], axis=1) # NEW
tts_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
tts_dfx2['SRC'] = 'Trip Duration'
tts_dfx2.drop(['Intercept', 'Group Var'], inplace=True)

### Average Sea Surface Temperature during Fishing Sets 

In [33]:
sst_md = smf.mixedlm("AVG_TEMP_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
sst_mdf = sst_md.fit()
#print(sst_mdf.summary())
sst_coef = pd.DataFrame(sst_mdf.params, columns = ['Coef'])
sst_p = pd.DataFrame(sst_mdf.pvalues, columns = ['Pval']) # new
sst_se = pd.DataFrame(sst_mdf.bse, columns = ['StdErr']) # NEW
sst_ci = pd.DataFrame(sst_mdf.conf_int())
sst_nobs = pd.DataFrame({'col1': sst_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
sst_df = pd.concat([sst_coef, sst_se, sst_p, sst_ci, sst_nobs], axis=1) # NEW
sst_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
sst_df['SRC'] = 'Average Set Sea Surface Temperature'
sst_df.drop(['Intercept', 'Group Var'], inplace=True)

sst_md2 = smf.mixedlm("AVG_TEMP_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
sst_mdf2 = sst_md2.fit()
sst_coef2 = pd.DataFrame(sst_mdf2.params, columns = ['Coef'])
sst_p2 = pd.DataFrame(sst_mdf2.pvalues, columns = ['Pval']) # new
sst_se2 = pd.DataFrame(sst_mdf2.bse, columns = ['StdErr']) # NEW
sst_ci2 = pd.DataFrame(sst_mdf2.conf_int())
sst_nobs2 = pd.DataFrame({'col1': sst_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
sst_df2 = pd.concat([sst_coef2, sst_se2, sst_p2, sst_ci2, sst_nobs2], axis=1) # NEW
sst_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
sst_df2['SRC'] = 'Average Set Sea Surface Temperature'
sst_df2.drop(['Intercept', 'Group Var'], inplace=True)

sst_mdx = smf.mixedlm("AVG_TEMP_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
sst_mdfx = sst_mdx.fit()
sst_coefx = pd.DataFrame(sst_mdfx.params, columns = ['Coef'])
sst_px = pd.DataFrame(sst_mdfx.pvalues, columns = ['Pval']) # new
sst_sex = pd.DataFrame(sst_mdfx.bse, columns = ['StdErr']) # NEW
sst_cix = pd.DataFrame(sst_mdfx.conf_int())
sst_nobsx = pd.DataFrame({'col1': sst_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
sst_dfx = pd.concat([sst_coefx, sst_sex, sst_px, sst_cix, sst_nobsx], axis=1) # NEW
sst_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
sst_dfx['SRC'] = 'Average Set Sea Surface Temperature'
sst_dfx.drop(['Intercept', 'Group Var'], inplace=True)

sst_mdx2 = smf.mixedlm("AVG_TEMP_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
sst_mdfx2 = sst_mdx2.fit()
sst_coefx2 = pd.DataFrame(sst_mdfx2.params, columns = ['Coef'])
sst_px2 = pd.DataFrame(sst_mdfx2.pvalues, columns = ['Pval']) # new
sst_sex2 = pd.DataFrame(sst_mdfx2.bse, columns = ['StdErr']) # NEW
sst_cix2 = pd.DataFrame(sst_mdfx2.conf_int())
sst_nobsx2 = pd.DataFrame({'col1': sst_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
sst_dfx2 = pd.concat([sst_coefx2, sst_sex2, sst_px2, sst_cix2, sst_nobsx2], axis=1) # NEW
sst_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
sst_dfx2['SRC'] = 'Average Set Sea Surface Temperature'
sst_dfx2.drop(['Intercept', 'Group Var'], inplace=True)

In [34]:
# Conbining all coef and CI for line plot
rspvar1 = cpue_df.append([lbs_df, rpue_df, set_df, dep_df, distshr_df, pptf_df, rev_df, trpdist_df, tts_df, sst_df, lat_df, long_df])

rspvar1['Coef'] = round(rspvar1.loc[:, 'Coef'] * 100, 2)
rspvar1['lowerCI'] = round(rspvar1.loc[:, 'lowerCI'] * 100, 2)
rspvar1['upperCI'] = round(rspvar1.loc[:, 'upperCI'] * 100, 2)
rspvar1['StdErr'] = round(rspvar1.loc[:, 'StdErr'] * 100, 2)

rspvar1['asymerror'] = rspvar1.loc[:, 'Coef'] - rspvar1.loc[:, 'lowerCI']


# Conbining all coef and CI for line plot
rspvar3 = cpue_df2.append([lbs_df2, rpue_df2, set_df2, dep_df2, distshr_df2, pptf_df2, rev_df2, trpdist_df2, tts_df2, sst_df2, lat_df2, long_df2])

rspvar3['Coef'] = round(rspvar3.loc[:, 'Coef'] * 100, 2)
rspvar3['lowerCI'] = round(rspvar3.loc[:, 'lowerCI'] * 100, 2)
rspvar3['upperCI'] = round(rspvar3.loc[:, 'upperCI'] * 100, 2)
rspvar3['StdErr'] = round(rspvar3.loc[:, 'StdErr'] * 100, 2)

rspvar3['asymerror'] = rspvar3.loc[:, 'Coef'] - rspvar3.loc[:, 'lowerCI']

# Conbining all coef and CI for line plot
rspvarx = cpue_dfx.append([lbs_dfx, rpue_dfx, set_dfx, dep_dfx, distshr_dfx, pptf_dfx, rev_dfx, trpdist_dfx, tts_dfx, sst_dfx, lat_dfx, long_dfx])

rspvarx['Coef'] = round(rspvarx.loc[:, 'Coef'] * 100, 2)
rspvarx['lowerCI'] = round(rspvarx.loc[:, 'lowerCI'] * 100, 2)
rspvarx['upperCI'] = round(rspvarx.loc[:, 'upperCI'] * 100, 2)
rspvarx['StdErr'] = round(rspvarx.loc[:, 'StdErr'] * 100, 2)

rspvarx['asymerror'] = rspvarx.loc[:, 'Coef'] - rspvarx.loc[:, 'lowerCI']


# Conbining all coef and CI for line plot
rspvarx4 = cpue_dfx2.append([lbs_dfx2, rpue_dfx2, set_dfx2, dep_dfx2, distshr_dfx2, pptf_dfx2, rev_dfx2, trpdist_dfx2, tts_dfx2, sst_dfx2, lat_dfx2, long_dfx2])

rspvarx4['Coef'] = round(rspvarx4.loc[:, 'Coef'] * 100, 2)
rspvarx4['lowerCI'] = round(rspvarx4.loc[:, 'lowerCI'] * 100, 2)
rspvarx4['upperCI'] = round(rspvarx4.loc[:, 'upperCI'] * 100, 2)
rspvarx4['StdErr'] = round(rspvarx4.loc[:, 'StdErr'] * 100, 2)

rspvarx4['asymerror'] = rspvarx4.loc[:, 'Coef'] - rspvarx4.loc[:, 'lowerCI']

In [35]:
print("\033[1m\n Fishery Metrics Results from Linear Mixed Efforts Model Season 1")
rspvar10 = rspvar1.set_index('SRC')
round(rspvar10.loc[:, ['Coef', 'StdErr', 'Pval', 'lowerCI', 'upperCI', 'Observations']], 3)


 Fishery Metrics Results from Linear Mixed Efforts Model Season 1


,Coef,StdErr,Pval,lowerCI,upperCI,Observations
SRC,,,,,,
CPUE,4.38,19.57,0.823,-33.97,42.73,79.0
Catch,17.91,22.42,0.424,-26.03,61.86,79.0
RPUE,3.76,22.71,0.869,-40.75,48.26,79.0
Effort,14.66,12.00,0.222,-8.86,38.17,79.0
Average Set Depth,23.42,11.02,0.034,1.82,45.02,79.0
Average Set Distance from Shore,18.89,12.31,0.125,-5.24,43.02,79.0
Proportion of Trip Fishing,8.67,7.85,0.269,-6.71,24.06,79.0
Ex-vessel Revenue,14.42,24.83,0.561,-34.24,63.08,79.0
Trip Distance,6.42,12.65,0.612,-18.37,31.20,79.0


In [36]:
print("\033[1m\n Fishery Metrics Results from Linear Mixed Efforts Model Season 2")
rspvar20 = rspvar3.set_index('SRC')
round(rspvar20.loc[:, ['Coef', 'StdErr',  'Pval', 'lowerCI', 'upperCI', 'Observations']], 3)


 Fishery Metrics Results from Linear Mixed Efforts Model Season 2


,Coef,StdErr,Pval,lowerCI,upperCI,Observations
SRC,,,,,,
CPUE,-7.54,14.98,0.615,-36.91,21.83,173.0
Catch,7.98,17.34,0.645,-26.00,41.96,173.0
RPUE,-4.93,16.47,0.765,-37.21,27.35,173.0
Effort,14.27,10.74,0.184,-6.77,35.32,173.0
Average Set Depth,-7.44,7.61,0.329,-22.36,7.49,173.0
Average Set Distance from Shore,4.83,8.68,0.578,-12.18,21.84,173.0
Proportion of Trip Fishing,2.01,5.60,0.720,-8.97,12.99,173.0
Ex-vessel Revenue,10.32,18.54,0.578,-26.02,46.65,173.0
Trip Distance,11.72,11.13,0.293,-10.10,33.54,173.0


In [37]:
print("\033[1m\n Fishery Metrics Results from Linear Mixed Efforts Model Season 1:\n Vessels that periodically carry observers vs Unobservable Vessels")
rspvar30 = rspvarx.set_index('SRC')
round(rspvar30.loc[:, ['Coef', 'StdErr',  'Pval', 'lowerCI', 'upperCI', 'Observations']], 3)


 Fishery Metrics Results from Linear Mixed Efforts Model Season 1:
 Vessels that periodically carry observers vs Unobservable Vessels


,Coef,StdErr,Pval,lowerCI,upperCI,Observations
SRC,,,,,,
CPUE,24.25,22.37,0.278,-19.58,68.09,136.0
Catch,40.62,35.21,0.249,-28.38,109.62,136.0
RPUE,4.93,23.31,0.833,-40.77,50.62,136.0
Effort,18.67,21.41,0.383,-23.30,60.64,136.0
Average Set Depth,36.98,14.77,0.012,8.03,65.93,136.0
Average Set Distance from Shore,39.05,15.70,0.013,8.27,69.83,136.0
Proportion of Trip Fishing,-15.36,10.80,0.155,-36.53,5.80,136.0
Ex-vessel Revenue,18.37,37.22,0.622,-54.59,91.32,136.0
Trip Distance,53.04,19.62,0.007,14.59,91.49,136.0


In [38]:
print("\033[1m\n Fishery Metrics Results from Linear Mixed Efforts Model Season 2:\n Vessels that periodically carry observers vs Unobservable Vessels")
rspvar40 = rspvarx4.set_index('SRC')
round(rspvar40.loc[:, ['Coef', 'StdErr',  'Pval', 'lowerCI', 'upperCI', 'Observations']], 3)


 Fishery Metrics Results from Linear Mixed Efforts Model Season 2:
 Vessels that periodically carry observers vs Unobservable Vessels


,Coef,StdErr,Pval,lowerCI,upperCI,Observations
SRC,,,,,,
CPUE,3.37,20.40,0.869,-36.62,43.35,228.0
Catch,7.24,26.40,0.784,-44.50,58.99,228.0
RPUE,-7.52,25.66,0.770,-57.82,42.78,228.0
Effort,7.02,17.77,0.693,-27.80,41.84,228.0
Average Set Depth,14.99,11.54,0.194,-7.63,37.60,228.0
Average Set Distance from Shore,3.54,8.88,0.690,-13.87,20.94,228.0
Proportion of Trip Fishing,-2.66,7.39,0.719,-17.14,11.83,228.0
Ex-vessel Revenue,-0.82,25.31,0.974,-50.43,48.78,228.0
Trip Distance,15.29,19.10,0.423,-22.14,52.72,228.0


## Mixed Effects Model: Set Level Analysis

In [39]:
# set level metrics
sql2 = "SELECT * FROM VMS.DGN_SETVSVMS_V3 WHERE landing_year IS NOT NULL AND prob_vessel = 0 AND is_bad_trip = 'F' AND is_bad_year = 'F'"
sets = pd.read_sql(sql2, con=connection)

In [40]:
# excluding 2013 through Jan 2014, as well as when trip start date is null
subset = sets.loc[(sets.loc[:, 'IS_BAD_YEAR'] == 'F') & (sets.loc[:, 'IS_BAD_TRIP'] == 'F') & (sets.loc[:, 'PROB_VESSEL'] == 0), :]
subset = subset.reset_index(drop=True) 

In [41]:
# The high limit for acceptable values is the mean plus 2 standard deviations    
LimitHiDur = np.mean(subset.loc[:, 'SET_DURATION_MINUTES']) + 2*np.std(subset.loc[:, 'SET_DURATION_MINUTES'])
LimitLoDur = np.mean(subset.loc[:, 'SET_DURATION_MINUTES']) - 2*np.std(subset.loc[:, 'SET_DURATION_MINUTES'])
limitsDur = (subset.loc[:, 'SET_DURATION_MINUTES'] >= LimitLoDur) & (subset.loc[:, 'SET_DURATION_MINUTES'] <= LimitHiDur)

In [42]:
print(LimitHiDur)
print(LimitLoDur)

1012.3966373631515
394.93177618408544


In [43]:
# removing all hms lbs outside of 2 standard deviations
setsSD_DUR = sets[limitsDur]

In [44]:
# removing all hms lbs outside of 2 standard deviations
setsSD_DURDist = setsSD_DUR.copy() #setsSD_DUR[limitsDist]

In [45]:
# Get names of indexes for which column trip 1295 and set 5
indexNames = setsSD_DURDist.loc[(setsSD_DURDist.loc[:, 'TRIP_NUM'] == 1295) & (setsSD_DURDist.loc[:, 'SET_NUM'] == 5), :].index
# Delete these row indexes from dataFrame
setsSD_DURDist.drop(indexNames , inplace=True)

In [46]:
#print(allsets.loc[:,'SET_NUM'].count())
print(setsSD_DUR.loc[:,'SET_NUM'].count())
print(setsSD_DURDist.loc[:,'SET_NUM'].count())

1785
1784


In [47]:
# converting depth longitude to positive values 
setsSD_DURDist['SET_AVG_DEPTH'] = setsSD_DURDist['SET_AVG_DEPTH'].abs()
setsSD_DURDist['SET_AVG_LONG'] = setsSD_DURDist['SET_AVG_LONG'].abs()

# converting vessel ID to a character field
setsSD_DURDist['VESSEL_NUM'] = setsSD_DURDist['VESSEL_NUM'].astype(str)


# log transform features
#setsSD_DURDist['SET_DURATION_MINUTES_LOG'] = np.log(setsSD_DURDist.loc[:,'SET_DURATION_MINUTES'])
setsSD_DURDist['SET_DISTANCE_METERS_LOG'] = np.log(setsSD_DURDist.loc[:,'SET_DISTANCE_METERS'])
setsSD_DURDist['SET_AVG_DIST_FROM_SHORE_LOG'] = np.log(setsSD_DURDist.loc[:,'SET_AVG_DIST_FROM_SHORE'])
setsSD_DURDist['SET_AVG_DEPTH_LOG'] = np.log(setsSD_DURDist.loc[:,'SET_AVG_DEPTH'])
setsSD_DURDist['SET_AVG_SST_LOG'] = np.log(setsSD_DURDist.loc[:,'SET_AVG_SST'])
setsSD_DURDist['SET_AVG_LONG_LOG'] = np.log(setsSD_DURDist.loc[:,'SET_AVG_LONG'])
setsSD_DURDist['SET_AVG_LAT_LOG'] = np.log(setsSD_DURDist.loc[:,'SET_AVG_LAT'])

In [48]:
# new column for presentations
setsSD_DURDist['Is Observed Trip'] = setsSD_DURDist['IS_OBSERVED_TRIP']

#decoding for plot presentations
Replace = setsSD_DURDist.loc[:, "Is Observed Trip"] == 1
setsSD_DURDist.loc[Replace, "Is Observed Trip"] = "True"

#decoding for plot presentations
Replace = setsSD_DURDist.loc[:, "Is Observed Trip"] == 0
setsSD_DURDist.loc[Replace, "Is Observed Trip"] = "False"


# new column for presentations
setsSD_DURDist['Is Observable Vessel'] = setsSD_DURDist['VESSEL_GROUP']

#decoding for plot presentations
Replace = setsSD_DURDist.loc[:, "Is Observable Vessel"] == 1
setsSD_DURDist.loc[Replace, "Is Observable Vessel"] = "True"

#decoding for plot presentations
Replace = setsSD_DURDist.loc[:, "Is Observable Vessel"] == 0
setsSD_DURDist.loc[Replace, "Is Observable Vessel"] = "False"

In [49]:
tripsum = setsSD_DURDist.copy()

In [50]:
# # only vessels that carry observers, quality trips (excluding 2013), and season 1 
tripsum_obs_2013noS1 = tripsum.loc[(tripsum.loc[:,"IS_OBS_VESSEL"] == 1) & (tripsum.loc[:,"IS_BAD_TRIP"] == 'F') & (tripsum.loc[:,"SEASON"] == 1), :]

 # only vessels that carry observers, quality trips (excluding 2013), and season 2
tripsum_obs_2013noS2 = tripsum.loc[(tripsum.loc[:,"IS_OBS_VESSEL"] == 1) & (tripsum.loc[:,"IS_BAD_TRIP"] == 'F') & (tripsum.loc[:,"SEASON"] == 2), :]


In [51]:
# # vessels that carry observers vs vessels that don't, quality trips (excluding 2013), and season 1
tripsum_obsunobsS1 = tripsum.loc[(tripsum.loc[:,"PROB_VESSEL"] == 0) & (tripsum.loc[:,"IS_BAD_TRIP"] == 'F') & (tripsum.loc[:,"SEASON"] == 1), :]

 # vessels that carry observers vs vessels that don't, quality trips (excluding 2013), and season 2 
tripsum_obsunobsS2 = tripsum.loc[(tripsum.loc[:,"PROB_VESSEL"] == 0) & (tripsum.loc[:,"IS_BAD_TRIP"] == 'F') & (tripsum.loc[:,"SEASON"] == 2), :]

In [52]:
print(tripsum_obs_2013noS1.loc[:, 'SET_NUM'].count())
print(tripsum_obs_2013noS2.loc[:, 'SET_NUM'].count())
print(tripsum_obsunobsS1.loc[:, 'SET_NUM'].count())
print(tripsum_obsunobsS2.loc[:, 'SET_NUM'].count())

396
859
654
1130


In [53]:
print("\033[1m\n Counts of Observervations: Observed vs Unobserved, Season 1")
print(tripsum_obs_2013noS1.loc[(tripsum_obs_2013noS1.loc[:,"IS_OBSERVED_TRIP"] == 1), "VESSEL_NUM"].count())
print(tripsum_obs_2013noS1.loc[(tripsum_obs_2013noS1.loc[:,"IS_OBSERVED_TRIP"] == 0), "VESSEL_NUM"].count())


 Counts of Observervations: Observed vs Unobserved, Season 1
145
251


In [54]:
print("\033[1m\n Counts of Observervations: Observed vs Unobserved, Season 2")
print(tripsum_obs_2013noS2.loc[(tripsum_obs_2013noS2.loc[:,"IS_OBSERVED_TRIP"] == 1), "VESSEL_NUM"].count())
print(tripsum_obs_2013noS2.loc[(tripsum_obs_2013noS2.loc[:,"IS_OBSERVED_TRIP"] == 0), "VESSEL_NUM"].count())


 Counts of Observervations: Observed vs Unobserved, Season 2
234
625


In [55]:
print("\033[1m\n Counts of Observervations: Observable vs Unobservable, Season 1")
print(tripsum_obsunobsS1.loc[(tripsum_obsunobsS1.loc[:,"IS_OBS_VESSEL"] == 1), "VESSEL_NUM"].count())
print(tripsum_obsunobsS1.loc[(tripsum_obsunobsS1.loc[:,"IS_OBS_VESSEL"] == 0), "VESSEL_NUM"].count())


 Counts of Observervations: Observable vs Unobservable, Season 1
396
258


In [56]:
print("\033[1m\n Counts of Observervations: Observable vs Unobservable, Season 2")
print(tripsum_obsunobsS2.loc[(tripsum_obsunobsS2.loc[:,"IS_OBS_VESSEL"] == 1), "VESSEL_NUM"].count())
print(tripsum_obsunobsS2.loc[(tripsum_obsunobsS2.loc[:,"IS_OBS_VESSEL"] == 0), "VESSEL_NUM"].count())


 Counts of Observervations: Observable vs Unobservable, Season 2
859
271


### Average Latitude of DGN Sets

In [57]:
lat_md = smf.mixedlm("SET_AVG_LAT_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
lat_mdf = lat_md.fit()
lat_coef = pd.DataFrame(lat_mdf.params, columns = ['Coef'])
lat_p = pd.DataFrame(lat_mdf.pvalues, columns = ['Pval']) # new
lat_se = pd.DataFrame(lat_mdf.bse, columns = ['StdErr']) # NEW
lat_ci = pd.DataFrame(lat_mdf.conf_int())
lat_nobs = pd.DataFrame({'col1': lat_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
lat_df = pd.concat([lat_coef, lat_se, lat_p, lat_ci, lat_nobs], axis=1) # NEW
lat_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True) # new
lat_df['SRC'] = 'Average Set Latitude'
lat_df.drop(['Intercept', 'Group Var'], inplace=True)

lat_md2 = smf.mixedlm("SET_AVG_LAT_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
lat_mdf2 = lat_md2.fit()
lat_coef2 = pd.DataFrame(lat_mdf2.params, columns = ['Coef'])
lat_p2 = pd.DataFrame(lat_mdf2.pvalues, columns = ['Pval']) # new
lat_se2 = pd.DataFrame(lat_mdf2.bse, columns = ['StdErr']) # NEW
lat_ci2 = pd.DataFrame(lat_mdf2.conf_int())
lat_nobs2 = pd.DataFrame({'col1': lat_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
lat_df2 = pd.concat([lat_coef2, lat_se2, lat_p2, lat_ci2, lat_nobs2], axis=1) # NEW
lat_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True) # new
lat_df2['SRC'] = 'Average Set Latitude'
lat_df2.drop(['Intercept', 'Group Var'], inplace=True)

lat_mdx = smf.mixedlm("SET_AVG_LAT_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
lat_mdfx = lat_mdx.fit()
lat_coefx = pd.DataFrame(lat_mdfx.params, columns = ['Coef'])
lat_px = pd.DataFrame(lat_mdfx.pvalues, columns = ['Pval']) # new
lat_sex = pd.DataFrame(lat_mdfx.bse, columns = ['StdErr']) # NEW
lat_cix = pd.DataFrame(lat_mdfx.conf_int())
lat_nobsx = pd.DataFrame({'col1': lat_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
lat_dfx = pd.concat([lat_coefx, lat_sex, lat_px, lat_cix, lat_nobsx], axis=1) # NEW
lat_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
lat_dfx['SRC'] = 'Average Set Latitude'
lat_dfx.drop(['Intercept', 'Group Var'], inplace=True)

lat_mdx2 = smf.mixedlm("SET_AVG_LAT_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
lat_mdfx2 = lat_mdx2.fit()
lat_coefx2 = pd.DataFrame(lat_mdfx2.params, columns = ['Coef'])
lat_px2 = pd.DataFrame(lat_mdfx2.pvalues, columns = ['Pval']) # new
lat_sex2 = pd.DataFrame(lat_mdfx2.bse, columns = ['StdErr']) # NEW
lat_cix2 = pd.DataFrame(lat_mdfx2.conf_int())
lat_nobsx2 = pd.DataFrame({'col1': lat_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
lat_dfx2 = pd.concat([lat_coefx2, lat_sex2, lat_px2, lat_cix2, lat_nobsx2], axis=1) # NEW
lat_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
lat_dfx2 ['SRC'] = 'Average Set Latitude'
lat_dfx2.drop(['Intercept', 'Group Var'], inplace=True)


### Average Longitude of DGN Sets

In [58]:

long_md = smf.mixedlm("SET_AVG_LONG_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
long_mdf = long_md.fit()
#print(long_mdf.summary())
long_coef = pd.DataFrame(long_mdf.params, columns = ['Coef'])
long_p = pd.DataFrame(long_mdf.pvalues, columns = ['Pval']) # new
long_se = pd.DataFrame(long_mdf.bse, columns = ['StdErr']) # NEW
long_ci = pd.DataFrame(long_mdf.conf_int())
long_nobs = pd.DataFrame({'col1': long_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
long_df = pd.concat([long_coef, long_se, long_p, long_ci, long_nobs], axis=1) # NEW
long_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
long_df['SRC'] = 'Average Set Longitude'
long_df.drop(['Intercept', 'Group Var'], inplace=True)

long_md2 = smf.mixedlm("SET_AVG_LONG_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
long_mdf2 = long_md2.fit()
long_coef2 = pd.DataFrame(long_mdf2.params, columns = ['Coef'])
long_p2 = pd.DataFrame(long_mdf2.pvalues, columns = ['Pval']) # new
long_se2 = pd.DataFrame(long_mdf2.bse, columns = ['StdErr']) # NEW
long_ci2 = pd.DataFrame(long_mdf2.conf_int())
long_nobs2 = pd.DataFrame({'col1': long_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
long_df2 = pd.concat([long_coef2, long_se2, long_p2, long_ci2, long_nobs2], axis=1) # NEW
long_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
long_df2['SRC'] = 'Average Set Longitude'
long_df2.drop(['Intercept', 'Group Var'], inplace=True)


long_mdx = smf.mixedlm("SET_AVG_LONG_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
long_mdfx = long_mdx.fit()
long_coefx = pd.DataFrame(long_mdfx.params, columns = ['Coef'])
long_px = pd.DataFrame(long_mdfx.pvalues, columns = ['Pval']) # new
long_sex = pd.DataFrame(long_mdfx.bse, columns = ['StdErr']) # NEW
long_cix = pd.DataFrame(long_mdfx.conf_int())
long_nobsx = pd.DataFrame({'col1': long_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
long_dfx = pd.concat([long_coefx, long_sex, long_px, long_cix, long_nobsx], axis=1) # NEW
long_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
long_dfx['SRC'] = 'Average Set Longitude'
long_dfx.drop(['Intercept', 'Group Var'], inplace=True)


long_mdx2 = smf.mixedlm("SET_AVG_LONG_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
long_mdfx2 = long_mdx2.fit()
long_coefx2 = pd.DataFrame(long_mdfx2.params, columns = ['Coef'])
long_px2 = pd.DataFrame(long_mdfx2.pvalues, columns = ['Pval']) # new
long_sex2 = pd.DataFrame(long_mdfx2.bse, columns = ['StdErr']) # NEW
long_cix2 = pd.DataFrame(long_mdfx2.conf_int())
long_nobsx2 = pd.DataFrame({'col1': long_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
long_dfx2 = pd.concat([long_coefx2, long_sex2, long_px2, long_cix2, long_nobsx2], axis=1) # NEW
long_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
long_dfx2 ['SRC'] = 'Average Set Longitude'
long_dfx2.drop(['Intercept', 'Group Var'], inplace=True)

### Average Bottom Depth of DGN Sets

In [59]:
dep_md = smf.mixedlm("SET_AVG_DEPTH_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
dep_mdf = dep_md.fit()
#print(dep_mdf.summary())
dep_coef = pd.DataFrame(dep_mdf.params, columns = ['Coef'])
dep_p = pd.DataFrame(dep_mdf.pvalues, columns = ['Pval']) # new
dep_se = pd.DataFrame(dep_mdf.bse, columns = ['StdErr']) # NEW
dep_ci = pd.DataFrame(dep_mdf.conf_int())
dep_nobs = pd.DataFrame({'col1': dep_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
dep_df = pd.concat([dep_coef, dep_se, dep_p, dep_ci, dep_nobs], axis=1) # NEW
dep_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
dep_df['SRC'] = 'Average Set Depth'
dep_df.drop(['Intercept', 'Group Var'], inplace=True)

dep_md2 = smf.mixedlm("SET_AVG_DEPTH_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
dep_mdf2 = dep_md2.fit()
dep_coef2 = pd.DataFrame(dep_mdf2.params, columns = ['Coef'])
dep_p2 = pd.DataFrame(dep_mdf2.pvalues, columns = ['Pval']) # new
dep_se2 = pd.DataFrame(dep_mdf2.bse, columns = ['StdErr']) # NEW
dep_ci2 = pd.DataFrame(dep_mdf2.conf_int())
dep_nobs2 = pd.DataFrame({'col1': dep_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
dep_df2 = pd.concat([dep_coef2, dep_se2, dep_p2, dep_ci2, dep_nobs2], axis=1) # NEW
dep_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
dep_df2['SRC'] = 'Average Set Depth'
dep_df2.drop(['Intercept', 'Group Var'], inplace=True)

dep_mdx = smf.mixedlm("SET_AVG_DEPTH_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
dep_mdfx = dep_mdx.fit()
dep_coefx = pd.DataFrame(dep_mdfx.params, columns = ['Coef'])
dep_px = pd.DataFrame(dep_mdfx.pvalues, columns = ['Pval']) # new
dep_sex = pd.DataFrame(dep_mdfx.bse, columns = ['StdErr']) # NEW
dep_cix = pd.DataFrame(dep_mdfx.conf_int())
dep_nobsx = pd.DataFrame({'col1': dep_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
dep_dfx = pd.concat([dep_coefx, dep_sex, dep_px, dep_cix, dep_nobsx], axis=1) # NEW
dep_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
dep_dfx['SRC'] = 'Average Set Depth'
dep_dfx.drop(['Intercept', 'Group Var'], inplace=True)

dep_mdx2 = smf.mixedlm("SET_AVG_DEPTH_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
dep_mdfx2 = dep_mdx2.fit()
dep_coefx2 = pd.DataFrame(dep_mdfx2.params, columns = ['Coef'])
dep_px2 = pd.DataFrame(dep_mdfx2.pvalues, columns = ['Pval']) # new
dep_sex2 = pd.DataFrame(dep_mdfx2.bse, columns = ['StdErr']) # NEW
dep_cix2 = pd.DataFrame(dep_mdfx2.conf_int())
dep_nobsx2 = pd.DataFrame({'col1': dep_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
dep_dfx2 = pd.concat([dep_coefx2, dep_sex2, dep_px2, dep_cix2, dep_nobsx2], axis=1) # NEW
dep_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
dep_dfx2['SRC'] = 'Average Set Depth'
dep_dfx2.drop(['Intercept', 'Group Var'], inplace=True)


### Average Distance from Shore of DGN Sets

In [60]:
distshr_md = smf.mixedlm("SET_AVG_DIST_FROM_SHORE_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
distshr_mdf = distshr_md.fit()
#print(distshr_mdf.summary())
distshr_coef = pd.DataFrame(distshr_mdf.params, columns = ['Coef'])
distshr_p = pd.DataFrame(distshr_mdf.pvalues, columns = ['Pval']) # new
distshr_se = pd.DataFrame(distshr_mdf.bse, columns = ['StdErr']) # NEW
distshr_ci = pd.DataFrame(distshr_mdf.conf_int())
distshr_nobs = pd.DataFrame({'col1': distshr_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
distshr_df = pd.concat([distshr_coef, distshr_se, distshr_p, distshr_ci, distshr_nobs], axis=1) # NEW
distshr_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
distshr_df['SRC'] = 'Average Set Distance from Shore'
distshr_df.drop(['Intercept', 'Group Var'], inplace=True)

distshr_md2 = smf.mixedlm("SET_AVG_DIST_FROM_SHORE_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
distshr_mdf2 = distshr_md2.fit()
#print(distshr_mdf2.summary())
distshr_coef2 = pd.DataFrame(distshr_mdf2.params, columns = ['Coef'])
distshr_p2 = pd.DataFrame(distshr_mdf2.pvalues, columns = ['Pval']) # new
distshr_se2 = pd.DataFrame(distshr_mdf2.bse, columns = ['StdErr']) # NEW
distshr_ci2 = pd.DataFrame(distshr_mdf2.conf_int())
distshr_nobs2 = pd.DataFrame({'col1': distshr_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
distshr_df2 = pd.concat([distshr_coef2, distshr_se2, distshr_p2, distshr_ci2, distshr_nobs2], axis=1) # NEW
distshr_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
distshr_df2['SRC'] = 'Average Set Distance from Shore'
distshr_df2.drop(['Intercept', 'Group Var'], inplace=True)

distshr_mdx = smf.mixedlm("SET_AVG_DIST_FROM_SHORE_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
distshr_mdfx = distshr_mdx.fit()
#print(distshr_mdf2.summary())
distshr_coefx = pd.DataFrame(distshr_mdfx.params, columns = ['Coef'])
distshr_px = pd.DataFrame(distshr_mdfx.pvalues, columns = ['Pval']) # new
distshr_sex = pd.DataFrame(distshr_mdfx.bse, columns = ['StdErr']) # NEW
distshr_cix = pd.DataFrame(distshr_mdfx.conf_int())
distshr_nobsx = pd.DataFrame({'col1': distshr_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
distshr_dfx = pd.concat([distshr_coefx, distshr_sex, distshr_px, distshr_cix, distshr_nobsx], axis=1) # NEW
distshr_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
distshr_dfx['SRC'] = 'Average Set Distance from Shore'
distshr_dfx.drop(['Intercept', 'Group Var'], inplace=True)

distshr_mdx2 = smf.mixedlm("SET_AVG_DIST_FROM_SHORE_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
distshr_mdfx2 = distshr_mdx2.fit()
#print(distshr_mdf2.summary())
distshr_coefx2 = pd.DataFrame(distshr_mdfx2.params, columns = ['Coef'])
distshr_px2 = pd.DataFrame(distshr_mdfx2.pvalues, columns = ['Pval']) # new
distshr_sex2 = pd.DataFrame(distshr_mdfx2.bse, columns = ['StdErr']) # NEW
distshr_cix2 = pd.DataFrame(distshr_mdfx2.conf_int())
distshr_nobsx2 = pd.DataFrame({'col1': distshr_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
distshr_dfx2 = pd.concat([distshr_coefx2, distshr_sex2, distshr_px2, distshr_cix2, distshr_nobsx2], axis=1) # NEW
distshr_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
distshr_dfx2['SRC'] = 'Average Set Distance from Shore'
distshr_dfx2.drop(['Intercept', 'Group Var'], inplace=True)


### Distance in Meters of DGN Sets

In [61]:
trpdist_md = smf.mixedlm("SET_DISTANCE_METERS_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
trpdist_mdf = trpdist_md.fit()
#print(trpdist_mdf.summary())
trpdist_coef = pd.DataFrame(trpdist_mdf.params, columns = ['Coef'])
trpdist_p = pd.DataFrame(trpdist_mdf.pvalues, columns = ['Pval']) # new
trpdist_se = pd.DataFrame(trpdist_mdf.bse, columns = ['StdErr']) # NEW
trpdist_ci = pd.DataFrame(trpdist_mdf.conf_int())
trpdist_nobs = pd.DataFrame({'col1': trpdist_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
trpdist_df = pd.concat([trpdist_coef, trpdist_se, trpdist_p, trpdist_ci, trpdist_nobs], axis=1) # NEW
trpdist_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
trpdist_df['SRC'] = 'Soak Time'
trpdist_df.drop(['Intercept', 'Group Var'], inplace=True)

trpdist_md2 = smf.mixedlm("SET_DISTANCE_METERS_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
trpdist_mdf2 = trpdist_md2.fit()
trpdist_coef2 = pd.DataFrame(trpdist_mdf2.params, columns = ['Coef'])
trpdist_p2 = pd.DataFrame(trpdist_mdf2.pvalues, columns = ['Pval']) # new
trpdist_se2 = pd.DataFrame(trpdist_mdf2.bse, columns = ['StdErr']) # NEW
trpdist_ci2 = pd.DataFrame(trpdist_mdf2.conf_int())
trpdist_nobs2 = pd.DataFrame({'col1': trpdist_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
trpdist_df2 = pd.concat([trpdist_coef2, trpdist_se2, trpdist_p2, trpdist_ci2, trpdist_nobs2], axis=1) # NEW
trpdist_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
trpdist_df2['SRC'] = 'Soak Time'
trpdist_df2.drop(['Intercept', 'Group Var'], inplace=True)

trpdist_mdx = smf.mixedlm("SET_DISTANCE_METERS_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
trpdist_mdfx = trpdist_mdx.fit()
#print(trpdist_mdfx.summary())
trpdist_coefx = pd.DataFrame(trpdist_mdfx.params, columns = ['Coef'])
trpdist_px = pd.DataFrame(trpdist_mdfx.pvalues, columns = ['Pval']) # new
trpdist_sex = pd.DataFrame(trpdist_mdfx.bse, columns = ['StdErr']) # NEW
trpdist_cix = pd.DataFrame(trpdist_mdfx.conf_int())
trpdist_nobsx = pd.DataFrame({'col1': trpdist_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
trpdist_dfx = pd.concat([trpdist_coefx, trpdist_sex, trpdist_px, trpdist_cix, trpdist_nobsx], axis=1) # NEW
trpdist_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
trpdist_dfx['SRC'] = 'Soak Time'
trpdist_dfx.drop(['Intercept', 'Group Var'], inplace=True)

trpdist_mdx2 = smf.mixedlm("SET_DISTANCE_METERS_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
trpdist_mdfx2 = trpdist_mdx2.fit()
#print(trpdist_mdfx.summary())
trpdist_coefx2 = pd.DataFrame(trpdist_mdfx2.params, columns = ['Coef'])
trpdist_px2 = pd.DataFrame(trpdist_mdfx2.pvalues, columns = ['Pval']) # new
trpdist_sex2 = pd.DataFrame(trpdist_mdfx2.bse, columns = ['StdErr']) # NEW
trpdist_cix2 = pd.DataFrame(trpdist_mdfx2.conf_int())
trpdist_nobsx2 = pd.DataFrame({'col1': trpdist_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
trpdist_dfx2 = pd.concat([trpdist_coefx2, trpdist_sex2, trpdist_px2, trpdist_cix2, trpdist_nobsx2], axis=1) # NEW
trpdist_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
trpdist_dfx2['SRC'] = 'Soak Time'
trpdist_dfx2.drop(['Intercept', 'Group Var'], inplace=True)


### Average Sea Surface Temperature of DGN Sets 

In [62]:
sst_md = smf.mixedlm("SET_AVG_SST_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS1, groups=tripsum_obs_2013noS1["VESSEL_NUM"])
sst_mdf = sst_md.fit()
#print(sst_mdf.summary())
sst_coef = pd.DataFrame(sst_mdf.params, columns = ['Coef'])
sst_p = pd.DataFrame(sst_mdf.pvalues, columns = ['Pval']) # new
sst_se = pd.DataFrame(sst_mdf.bse, columns = ['StdErr']) # NEW
sst_ci = pd.DataFrame(sst_mdf.conf_int())
sst_nobs = pd.DataFrame({'col1': sst_mdf.nobs}, index=['IS_OBSERVED_TRIP']) # new
sst_df = pd.concat([sst_coef, sst_se, sst_p, sst_ci, sst_nobs], axis=1) # NEW
sst_df.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
sst_df['SRC'] = 'Average Set Sea Surface Temperature'
sst_df.drop(['Intercept', 'Group Var'], inplace=True)

sst_md2 = smf.mixedlm("SET_AVG_SST_LOG ~ IS_OBSERVED_TRIP", tripsum_obs_2013noS2, groups=tripsum_obs_2013noS2["VESSEL_NUM"])
sst_mdf2 = sst_md2.fit()
sst_coef2 = pd.DataFrame(sst_mdf2.params, columns = ['Coef'])
sst_p2 = pd.DataFrame(sst_mdf2.pvalues, columns = ['Pval']) # new
sst_se2 = pd.DataFrame(sst_mdf2.bse, columns = ['StdErr']) # NEW
sst_ci2 = pd.DataFrame(sst_mdf2.conf_int())
sst_nobs2 = pd.DataFrame({'col1': sst_mdf2.nobs}, index=['IS_OBSERVED_TRIP']) # new
sst_df2 = pd.concat([sst_coef2, sst_se2, sst_p2, sst_ci2, sst_nobs2], axis=1) # NEW
sst_df2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
sst_df2['SRC'] = 'Average Set Sea Surface Temperature'
sst_df2.drop(['Intercept', 'Group Var'], inplace=True)

sst_mdx = smf.mixedlm("SET_AVG_SST_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS1, groups=tripsum_obsunobsS1["VESSEL_NUM"])
sst_mdfx = sst_mdx.fit()
sst_coefx = pd.DataFrame(sst_mdfx.params, columns = ['Coef'])
sst_px = pd.DataFrame(sst_mdfx.pvalues, columns = ['Pval']) # new
sst_sex = pd.DataFrame(sst_mdfx.bse, columns = ['StdErr']) # NEW
sst_cix = pd.DataFrame(sst_mdfx.conf_int())
sst_nobsx = pd.DataFrame({'col1': sst_mdfx.nobs}, index=['IS_OBS_VESSEL']) # new
sst_dfx = pd.concat([sst_coefx, sst_sex, sst_px, sst_cix, sst_nobsx], axis=1) # NEW
sst_dfx.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
sst_dfx['SRC'] = 'Average Set Sea Surface Temperature'
sst_dfx.drop(['Intercept', 'Group Var'], inplace=True)

sst_mdx2 = smf.mixedlm("SET_AVG_SST_LOG ~ IS_OBS_VESSEL", tripsum_obsunobsS2, groups=tripsum_obsunobsS2["VESSEL_NUM"])
sst_mdfx2 = sst_mdx2.fit()
sst_coefx2 = pd.DataFrame(sst_mdfx2.params, columns = ['Coef'])
sst_px2 = pd.DataFrame(sst_mdfx2.pvalues, columns = ['Pval']) # new
sst_sex2 = pd.DataFrame(sst_mdfx2.bse, columns = ['StdErr']) # NEW
sst_cix2 = pd.DataFrame(sst_mdfx2.conf_int())
sst_nobsx2 = pd.DataFrame({'col1': sst_mdfx2.nobs}, index=['IS_OBS_VESSEL']) # new
sst_dfx2 = pd.concat([sst_coefx2, sst_sex2, sst_px2, sst_cix2, sst_nobsx2], axis=1) # NEW
sst_dfx2.rename(index=str, columns={'Coef': 'Coef', 0 : 'lowerCI', 1 : 'upperCI', 'col1' : 'Observations'}, inplace=True)
sst_dfx2['SRC'] = 'Average Set Sea Surface Temperature'
sst_dfx2.drop(['Intercept', 'Group Var'], inplace=True)




In [63]:
# Conbining all coef and CI for line plot
rspvar1 = dep_df.append([distshr_df, trpdist_df, sst_df, lat_df, long_df])

rspvar1['Coef'] = round(rspvar1.loc[:, 'Coef'] * 100, 2)
rspvar1['lowerCI'] = round(rspvar1.loc[:, 'lowerCI'] * 100, 2)
rspvar1['upperCI'] = round(rspvar1.loc[:, 'upperCI'] * 100, 2)
rspvar1['StdErr'] = round(rspvar1.loc[:, 'StdErr'] * 100, 2)

rspvar1['asymerror'] = rspvar1.loc[:, 'Coef'] - rspvar1.loc[:, 'lowerCI']


# Conbining all coef and CI for line plot
rspvar3 = dep_df2.append([distshr_df2, trpdist_df2, sst_df2, lat_df2, long_df2])

rspvar3['Coef'] = round(rspvar3.loc[:, 'Coef'] * 100, 2)
rspvar3['lowerCI'] = round(rspvar3.loc[:, 'lowerCI'] * 100, 2)
rspvar3['upperCI'] = round(rspvar3.loc[:, 'upperCI'] * 100, 2)
rspvar3['StdErr'] = round(rspvar3.loc[:, 'StdErr'] * 100, 2)

rspvar3['asymerror'] = rspvar3.loc[:, 'Coef'] - rspvar3.loc[:, 'lowerCI']

# Conbining all coef and CI for line plot
rspvarx = dep_dfx.append([distshr_dfx, trpdist_dfx, sst_dfx, lat_dfx, long_dfx])

rspvarx['Coef'] = round(rspvarx.loc[:, 'Coef'] * 100, 2)
rspvarx['lowerCI'] = round(rspvarx.loc[:, 'lowerCI'] * 100, 2)
rspvarx['upperCI'] = round(rspvarx.loc[:, 'upperCI'] * 100, 2)
rspvarx['StdErr'] = round(rspvarx.loc[:, 'StdErr'] * 100, 2)

rspvarx['asymerror'] = rspvarx.loc[:, 'Coef'] - rspvarx.loc[:, 'lowerCI']


# Conbining all coef and CI for line plot
rspvarx4 = dep_dfx2.append([distshr_dfx2, trpdist_dfx2, sst_dfx2, lat_dfx2, long_dfx2])

rspvarx4['Coef'] = round(rspvarx4.loc[:, 'Coef'] * 100, 2)
rspvarx4['lowerCI'] = round(rspvarx4.loc[:, 'lowerCI'] * 100, 2)
rspvarx4['upperCI'] = round(rspvarx4.loc[:, 'upperCI'] * 100, 2)
rspvarx4['StdErr'] = round(rspvarx4.loc[:, 'StdErr'] * 100, 2)

rspvarx4['asymerror'] = rspvarx4.loc[:, 'Coef'] - rspvarx4.loc[:, 'lowerCI']

In [64]:
print("\033[1m\n Fishery Metrics Results from Linear Mixed Efforts Model Season 1")
rspvar10 = rspvar1.set_index('SRC')
round(rspvar10.loc[:, ['Coef', 'StdErr', 'Pval', 'lowerCI', 'upperCI', 'Observations']], 3)


 Fishery Metrics Results from Linear Mixed Efforts Model Season 1


,Coef,StdErr,Pval,lowerCI,upperCI,Observations
SRC,,,,,,
Average Set Depth,16.61,6.49,0.010,3.90,29.33,396.0
Average Set Distance from Shore,10.26,6.14,0.095,-1.77,22.30,396.0
Soak Time,11.85,4.81,0.014,2.42,21.28,396.0
Average Set Sea Surface Temperature,-1.02,0.96,0.286,-2.90,0.86,396.0
Average Set Latitude,-1.12,0.85,0.189,-2.79,0.55,396.0
Average Set Longitude,0.04,0.19,0.824,-0.32,0.40,396.0


In [65]:
print("\033[1m\n Fishery Metrics Results from Linear Mixed Efforts Model Season 2")
rspvar20 = rspvar3.set_index('SRC')
round(rspvar20.loc[:, ['Coef', 'StdErr', 'Pval', 'lowerCI', 'upperCI', 'Observations']], 3)


 Fishery Metrics Results from Linear Mixed Efforts Model Season 2


,Coef,StdErr,Pval,lowerCI,upperCI,Observations
SRC,,,,,,
Average Set Depth,-4.78,4.10,0.243,-12.81,3.24,859.0
Average Set Distance from Shore,-8.68,3.87,0.025,-16.27,-1.09,859.0
Soak Time,15.39,3.13,0.000,9.25,21.53,859.0
Average Set Sea Surface Temperature,0.98,0.43,0.021,0.15,1.82,859.0
Average Set Latitude,-0.08,0.26,0.751,-0.60,0.43,859.0
Average Set Longitude,-0.17,0.10,0.095,-0.37,0.03,859.0


In [66]:
print("\033[1m\n Fishery Metrics Results from Linear Mixed Efforts Model Season 1:\n Vessels that periodically carry observers vs Unobservable Vessels")
rspvar30 = rspvarx.set_index('SRC')
round(rspvar30.loc[:, ['Coef', 'StdErr', 'Pval', 'lowerCI', 'upperCI', 'Observations']], 3)


 Fishery Metrics Results from Linear Mixed Efforts Model Season 1:
 Vessels that periodically carry observers vs Unobservable Vessels


,Coef,StdErr,Pval,lowerCI,upperCI,Observations
SRC,,,,,,
Average Set Depth,50.57,16.15,0.002,18.92,82.21,654.0
Average Set Distance from Shore,38.17,17.17,0.026,4.52,71.83,654.0
Soak Time,9.13,7.43,0.219,-5.44,23.69,654.0
Average Set Sea Surface Temperature,-4.70,2.58,0.068,-9.75,0.36,654.0
Average Set Latitude,0.79,1.23,0.522,-1.62,3.19,654.0
Average Set Longitude,0.53,0.38,0.156,-0.20,1.27,654.0


In [67]:
print("\033[1m\n Fishery Metrics Results from Linear Mixed Efforts Model Season 2:\n Vessels that periodically carry observers vs Unobservable Vessels")
rspvar40 = rspvarx4.set_index('SRC')
round(rspvar40.loc[:, ['Coef', 'StdErr', 'Pval', 'lowerCI', 'upperCI', 'Observations']], 3)


 Fishery Metrics Results from Linear Mixed Efforts Model Season 2:
 Vessels that periodically carry observers vs Unobservable Vessels


,Coef,StdErr,Pval,lowerCI,upperCI,Observations
SRC,,,,,,
Average Set Depth,28.10,14.00,0.045,0.65,55.55,1130.0
Average Set Distance from Shore,15.10,9.77,0.122,-4.05,34.24,1130.0
Soak Time,-5.70,7.59,0.453,-20.58,9.18,1130.0
Average Set Sea Surface Temperature,-0.80,1.82,0.660,-4.38,2.77,1130.0
Average Set Latitude,0.28,0.77,0.720,-1.23,1.79,1130.0
Average Set Longitude,0.21,0.35,0.556,-0.48,0.89,1130.0
